# Another implementation of word2vec: this is the first version that I saw

In [5]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import inspect
import string
import time


In [29]:
timing = {}

def get_time(f,args):
    key =  f.__name__  + "-" +"_".join([str(arg) for arg in args])
    return timing[key]

def timeit(method):

    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        timed.__name__ = method.__name__        
        te = time.time()
        fkey = method.__name__  + "-" +"_".join([str(arg) for arg in args])
        timing[fkey] = te-ts

        #print('%r (%r, %r) %2.2f sec'% (method.__name__, args, kw, te-ts))
        return result

    return timed

class Foo(object):

    @timeit
    def foo(self, a=2, b=3):
        time.sleep(0.2)

@timeit
def f1(a,b):
    return a+b

f1(1,2)

3

In [30]:
get_time(f1,[1,2])

1.430511474609375e-06

In [33]:
filename = 'pt96.txt'  
file_path = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
file_path = os.path.join(file_path,'data')
file_path = os.path.join(file_path,filename)

@timeit
def read_data(file_path):
    dic_trans = {key: None for key in string.punctuation}
    translator = str.maketrans(dic_trans)
    words = []
    with open(file_path) as inputfile:
        for line in inputfile:
            aux = line.translate(translator)
            words.extend(aux.strip().split())
    return words

words = read_data(file_path)
print('Data size', len(words))

Data size 15164437


In [34]:
timing

{'f1-1_2': 1.430511474609375e-06,
 'read_data-/home/felipe/Desktop/word2vec-basic/source/data/pt96.txt': 8.594845056533813}

In [37]:
get_time(read_data,[file_path])

8.594845056533813

In [9]:
vocabulary_size = 50000

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
del words  # Hint to reduce memory.

Most common words (+UNK) [['UNK', 1077275], ('de', 763544), ('e', 422852), ('a', 422319), ('o', 335045)]
Sample data [7825, 18650, 0, 5514, 18650, 0, 9977, 6489, 8555, 2190]


In [10]:
data_index = 0

def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(int(batch_size / num_skips)):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], '->', labels[i, 0])
  print(reverse_dictionary[batch[i]], '->', reverse_dictionary[labels[i, 0]])

18650 -> 0
Contos -> UNK
18650 -> 7825
Contos -> Conto
0 -> 5514
UNK -> 1870
0 -> 18650
UNK -> Contos
5514 -> 18650
1870 -> Contos
5514 -> 0
1870 -> UNK
18650 -> 0
Contos -> UNK
18650 -> 5514
Contos -> 1870


In [11]:
batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()
with graph.as_default():
    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
    # Variables.
    embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    softmax_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                         stddev=1.0 / math.sqrt(embedding_size)))
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
  
    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(tf.nn.sampled_softmax_loss(softmax_weights, softmax_biases,train_labels, embed,num_sampled, vocabulary_size))

    # Optimizer.
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
  
    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings))

In [12]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  average_loss = 0
  for step in range(num_steps):
    batch_data, batch_labels = generate_batch(
      batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step", step, ":", average_loss)
      average_loss = 0
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = "Nearest to %s:" % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = "%s %s," % (log, close_word)
        print(log)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0 : 7.24202442169
Nearest to é: Mor, Mortensen, Juvenis, Católicas, Bot, Diretrizes, abrupta, preservada,
Nearest to com: 160, recorre, Stanislaw, Prudente, Katie, vermelhos, Saíram, Dolorosa,
Nearest to mas: séria, 249, raptor, explorava, camisas, síria, kgkg, pilhas,
Nearest to o: Mitsubishi, calorífico, laborais, Prússia, 54°, Paget, colinas, Iniciando,
Nearest to da: evangelizar, cedo, Alt, Elenco, ser, Inferno, abandonadas, Caitlyn,
Nearest to a: Third, Ennio, energéticas, Josefa, Musashi, tático, dependerá, 2005,
Nearest to nas: cabida, trimestre, apagamento, outros1, insurgentes, publicadores, Nicol, Punho,
Nearest to pode: Ariano, realizando, sujeitos, faísca, Boyz, Zeratul, inverno, insectos,
Nearest to entre: Type, Corvo, Monetária, inventaram, apontamentos, Powers, casarão, spins,
Nearest to ter: Abecásia, paciência, paciente, Germânica, extremidades, escrevente, aliado, cause,
Nearest to ao: Introduction, ingresso, participava, resolutamente

In [45]:
from scipy import spatial

def analogy(w1,w2,w3,index_to_word,word_to_index, embeddings):
    a = embeddings[word_to_index[w1]]
    b = embeddings[word_to_index[w2]]
    c = embeddings[word_to_index[w3]]
    def apply_dot(x):
        return b.dot(x) - a.dot(x) + c.dot(x)
    all_results = [(apply_dot(w), index) for index, w in enumerate(embeddings) if (index!= word_to_index[w1] and index!= word_to_index[w2] and index!= word_to_index[w3])]
    all_results.sort(reverse=True)
    result = [(index_to_word[index],value) for (value,index) in all_results[0:11]]
    return result[0][0]  

def top_k_sim(vector, index_to_word, embeddings,k=10):
    all_sim = [(1 - spatial.distance.cosine(vector, w),index) for index, w in enumerate(embeddings)]
    all_sim.sort(reverse =True)
    result = [(index_to_word[index],value) for (value,index) in all_sim[0:k+1]]
    return result 

In [33]:
mulher = final_embeddings[dictionary['mulher']]
top_k_sim(mulher,reverse_dictionary,final_embeddings)

[('mulher', 0.99999994039535522),
 ('moça', 0.4613415002822876),
 ('filha', 0.45169356465339661),
 ('Bednorz', 0.39365583658218384),
 ('mãe', 0.38549751043319702),
 ('tia', 0.37841418385505676),
 ('carta', 0.37613511085510254),
 ('visita', 0.37451835718860371),
 ('Heloísa', 0.36902861368315054),
 ('derrubado', 0.35476463167456584),
 ('Ignaz', 0.35475873081438136)]

In [46]:
analogy("homem","mulher","rei",reverse_dictionary,dictionary,final_embeddings)

'Crockett'

In [44]:
eval_file="analogies_pt.txt"
vocab = set(dictionary.keys())
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
prefix = os.path.join(currentdir, "evaluation")
file_path = os.path.join(prefix, eval_file)
valid_tests = 0
correct_answer = 0
total_lines = 0
total_loss = 0


In [ ]:
import time
import sys
initial_time = time.time()
with open(file_path) as inputfile:
    for line in inputfile:
        total_lines += 1
        initial = time.time()
        list_line = line.strip().split()
        if all([word in vocab for word in list_line]):
            valid_tests += 1
            analogue = analogy(list_line[0],
                               list_line[1],
                               list_line[2],
                               reverse_dictionary,
                               dictionary,
                               final_embeddings)
            print("\nAnalogy -->", list_line)
            print("prediction -->", analogue)
            if analogue == list_line[3]:
                correct_answer += 1
                print("\nYESSSSSSSSSSSSSSSSSSSSS\n")
            current_time = time.time() - initial
            sys.stdout.write('\rcurrent_line:{}, duration = {}'.format(total_lines,
                                                                        current_time))
            sys.stdout.flush()

print("\ntotal_lines = {}".format(total_lines))
print("valid_tests = {}".format(valid_tests))
print("correct_answer = {}".format(correct_answer))
print("total_loss = {}".format(total_loss))
print("duration = {}".format(time.time() - initial_time))



Analogy --> ['argentina', 'peso', 'europa', 'euro']
prediction --> Standard
current_line:25470, duration = 0.17199397087097168
Analogy --> ['argentina', 'peso', 'índia', 'rupia']
prediction --> presenças
current_line:25472, duration = 0.17736601829528809
Analogy --> ['europa', 'euro', 'índia', 'rupia']
prediction --> desporto
current_line:25694, duration = 0.17482328414916992
Analogy --> ['europa', 'euro', 'argentina', 'peso']
prediction --> psicóloga
current_line:25714, duration = 0.1721484661102295
Analogy --> ['índia', 'rupia', 'argentina', 'peso']
prediction --> querubins
current_line:25770, duration = 0.17726445198059082
Analogy --> ['índia', 'rupia', 'europa', 'euro']
prediction --> 153
current_line:25778, duration = 0.17674016952514648
Analogy --> ['rapaz', 'moça', 'irmãos', 'irmãs']
prediction --> abrigava
current_line:28737, duration = 0.17703747749328613
Analogy --> ['rapaz', 'moça', 'pai', 'mãe']
prediction --> mãe

YESSSSSSSSSSSSSSSSSSSSS

current_line:28738, duration = 0.

In [24]:
reverse_dictionary[414]

'mulher'